In [1]:
import os
import pandas as pd
import subprocess
import re
DATA_IN_PATH = './data/'

#해당 디렉토리에서 txt파일 읽고 dict에 저장하기
def read_file(directory):
    scripts = {}
    scripts["name"] = []  #내가 보기 편하게 하기 위해 넣어논거 나중에 제거할 예정 , encoding='UTF8'
    scripts["code"] = []
    for file in os.listdir(directory):
        if 'txt' in file:
            try:
                with open(os.path.join(directory ,file), "r", encoding='UTF8') as script:
                    scripts["name"].append(file)
                    code = script.readlines()
                    scripts["code"].append(preprocess(code)) #preprocess(code)
            except UnicodeDecodeError as e:
                scripts["code"].append(None)
    return pd.DataFrame.from_dict(scripts)


def preprocess(code):
    result = []
    multi = 0
    for line in code:
        line = re.sub("[^a-zA-Z0-9-._]", " ", line)
        #언더 바 두개 이상으로 붙어있는거 제거
        result.append(re.sub("__+", " ", line))
    tmp = ' '.join(result) #하나의 문자열로 합치기
    return tmp


#디렉토리 안에 있는 ps1파일 psparser실행 -> txt로 결과값 저장
def run_psparser(directory):
    #PATH에 자기가 위치하고 있는 곳 경로 추가 
    curpath = os.getcwd() + ';'
    env = os.environ
    path = curpath+env['PATH']
    env['PATH'] = path
    #psParser.exe는 파이썬 코드와 같은 위치에 있어야함
    result = subprocess.run(['psParser.exe', directory])

def labeling(directory): #라벨링 작업
    malicious_df = read_file(os.path.join(directory,"malicious/result"))
    clean_df = read_file(os.path.join(directory,"clean/result"))
    malicious_df["sentiment"] = 1 
    clean_df["sentiment"] = 0
    return pd.concat([malicious_df, clean_df],ignore_index=True)
    
